In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
max_text_len=100
max_summary_len=20

x_voc = 25545
y_voc = 7629

In [3]:
from keras import backend as K 
import gensim
from numpy import *
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

print("Size of vocabulary from the w2v model = {}".format(x_voc))

K.clear_session()

latent_dim = 300
embedding_dim=200

encoder_inputs = Input(shape=(max_text_len,))

enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output3, state_h3, state_c3= encoder_lstm3(encoder_output2)

encoder_lstm4=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output3)


decoder_inputs = Input(shape=(None,))

dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()


Using TensorFlow backend.


Size of vocabulary from the w2v model = 25545
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 200)     5109000     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100, 300), ( 601200      embedding[0][0]                  
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 100, 300), ( 721200      lstm[0][0]                       
________________________________________________

In [4]:
model = load_model("../input/nlp100lstmrmspropweights/nlp_100_weights.h5py")

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 200)     5109000     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100, 300), ( 601200      embedding[0][0]                  
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 100, 300), ( 721200      lstm[0][0]                       
______________________________________________________________________________________________

In [8]:
import re

def text_strip(row):
        
    row=re.sub("(\\t)", ' ', str(row)).lower() #remove escape charecters
    row=re.sub("(\\r)", ' ', str(row)).lower() 
    row=re.sub("(\\n)", ' ', str(row)).lower()

    row=re.sub("(__+)", ' ', str(row)).lower()   #remove _ if it occors more than one time consecutively
    row=re.sub("(--+)", ' ', str(row)).lower()   #remove - if it occors more than one time consecutively
    row=re.sub("(~~+)", ' ', str(row)).lower()   #remove ~ if it occors more than one time consecutively
    row=re.sub("(\+\++)", ' ', str(row)).lower()   #remove + if it occors more than one time consecutively
    row=re.sub("(\.\.+)", ' ', str(row)).lower()   #remove . if it occors more than one time consecutively

    row=re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", ' ', str(row)).lower() #remove <>()|&©ø"',;?~*!

    row=re.sub("(mailto:)", ' ', str(row)).lower() #remove mailto:
    row=re.sub(r"(\\x9\d)", ' ', str(row)).lower() #remove \x9* in text
    row=re.sub("([iI][nN][cC]\d+)", 'INC_NUM', str(row)).lower() #replace INC nums to INC_NUM
    row=re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)", 'CM_NUM', str(row)).lower() #replace CM# and CHG# to CM_NUM


    row=re.sub("(\.\s+)", ' ', str(row)).lower() #remove full stop at end of words(not between)
    row=re.sub("(\-\s+)", ' ', str(row)).lower() #remove - at end of words(not between)
    row=re.sub("(\:\s+)", ' ', str(row)).lower() #remove : at end of words(not between)

    row=re.sub("(\s+.\s+)", ' ', str(row)).lower() #remove any single charecters hanging between 2 spaces

    try:
        url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', str(row))
        repl_url = url.group(3)
        row = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)',repl_url, str(row))
    except:
        pass #there might be emails with no url in them



    row = re.sub("(\s+)",' ',str(row)).lower() #remove multiple spaces

    row=re.sub("(\s+.\s+)", ' ', str(row)).lower() #remove any single charecters hanging between 2 spaces

    return row

In [9]:
from keras_preprocessing.text import tokenizer_from_json
import io, json

with open('../input/nlp-tokenizers/x_tokenizer.json') as f:
    data = json.load(f)
    x_tokenizer = tokenizer_from_json(data)

    
with open('../input/nlp-tokenizers/y_tokenizer.json') as f:
    data = json.load(f)
    y_tokenizer = tokenizer_from_json(data)

reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [10]:
def preProcess(new_inp):
    strip_inp = text_strip(new_inp)

    final_inp = x_tokenizer.texts_to_sequences([strip_inp])
    final_inp = pad_sequences(final_inp,  maxlen=max_text_len, padding='post')

    return final_inp[0]


In [12]:
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

dec_emb2= dec_emb_layer(decoder_inputs) 
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

decoder_outputs2 = decoder_dense(decoder_outputs2) 

decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [13]:
def decode_sequence(input_seq):
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1,1))
    
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        e_h, e_c = h, c

    return decoded_sentence

In [14]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [15]:
new_inp = "South Korean smartphone maker Samsung has unveiled new black variant of its Galaxy S7 Edge smartphone called pearl black which looks similar to the jet black iphone variant. The colour will be exclusive to the 128 gb version of the device and will be available to buy in select markets from december 9"
final_inp = preProcess(new_inp)
print("Review:", seq2text(final_inp))
print("Predicted summary:",decode_sequence(final_inp.reshape(1,max_text_len)))

Review: south korean smartphone maker samsung has unveiled new black variant of its galaxy s7 edge smartphone called pearl black which looks similar to the jet black iphone variant the colour will be exclusive to the 128 gb version of the device and will be available to buy in select markets from december 9 
Predicted summary: start samsung unveils galaxy s7 edge in one end


In [16]:
new_inp = "former delhi bjp mla vijay jolly has been booked for rape after woman alleged that he sexually assaulted her police said on thursday the delhi-based woman accused the former legislator of spiking her drink before sexually assaulting her in gurugram resort jolly had also filed police complaint against the woman accusing her of blackmailing and extortion."
final_inp = preProcess(new_inp)
print("Review:", seq2text(final_inp))
print("Predicted summary:",decode_sequence(final_inp.reshape(1,max_text_len)))

Review: former delhi bjp mla vijay jolly has been booked for rape after woman alleged that he sexually assaulted her police said on thursday the delhi based woman accused the former legislator of spiking her drink before sexually assaulting her in gurugram resort jolly had also filed police complaint against the woman accusing her of blackmailing and extortion 
Predicted summary: start bjp mla booked for 39 39 end


In [17]:
new_inp = "india #39 top-ranked shuttler pv sindhu made winning start to the world superseries finals in dubai after winning her opening round match against world number eight akane yamaguchi on wednesday the rio silver medal winning badminton player beat yamaguchi 12-21 21-8 21-15 in the group match sindhu will next face china #39 sun yu in her next group game."
final_inp = preProcess(new_inp)
print("Review:", seq2text(final_inp))
print("Predicted summary:",decode_sequence(final_inp.reshape(1,max_text_len)))

Review: india 39 top ranked shuttler pv sindhu made winning start to the world superseries finals in dubai after winning her opening round match against world number eight yamaguchi on wednesday the rio silver medal winning badminton player beat yamaguchi 12 21 21 8 21 15 in the group match sindhu will next face china 39 sun yu in her next group game 
Predicted summary: start india wins silver at asian championships end


In [18]:
new_inp = "a three-minute video explains how smoke is caused by small particles floating in the air and how pollution affects the human body detailing the standard measurements to analyse pollution the video adds that air pollution is linked to health problems like heart disease stroke and autism it further explains how particles can get into the lungs and cause serious damage."
final_inp = preProcess(new_inp)
print("Review:", seq2text(final_inp))
print("Predicted summary:",decode_sequence(final_inp.reshape(1,max_text_len)))

Review: a three minute video explains how smoke is caused by small particles floating in the air and how pollution affects the human body detailing the standard measurements to analyse pollution the video adds that air pollution is linked to health problems like heart disease stroke and autism it further explains how particles can get into the lungs and cause serious damage 
Predicted summary: start video explains why we can 39 be end


In [19]:
new_inp = "alastair cook became the first england player to break into the top ten run-getters in test cricket history after scoring 12 runs in the second innings against india in the third test cook who now has 10 934 runs in 249 test innings overtook former australian captain steve waugh who had scored 10 927 runs in his test career."
final_inp = preProcess(new_inp)
print("Review:", seq2text(final_inp))
print("Predicted summary:",decode_sequence(final_inp.reshape(1,max_text_len)))

Review: alastair cook became the first england player to break into the top ten run in test cricket history after scoring 12 runs in the second innings against india in the third test cook who now has 10 runs in 249 test innings overtook former australian captain steve waugh who had scored 10 runs in his test career 
Predicted summary: start ashwin 39 first ever test ton helps australia beat england end
